# Factory Planning II

## Objective and Prerequisites

Are you up for a major production planning challenge? Try this example where you will learn how to create an optimal production plan that will not only maximize profits, but also determine which month in which to perform maintenance operations on your machines.

More information on this type of model can be found in example #4 of the fifth edition of Modeling Building in Mathematical Programming by H. P. Williams on pages 256 and 302 – 303.

This modeling example is at the intermediate level, where we assume that you know Python and are familiar with the Gurobi Python API. In addition, you should have some knowledge about building mathematical optimization models.

**Download the Repository** <br />
You can download the repository containing this and other examples by clicking [here](https://github.com/Gurobi/modeling-examples/archive/master.zip).

---
## Problem Description

A factory makes seven products (Prod 1 to Prod 7) using a range of machines including:

- Four grinders
- Two vertical drills
- Three horizontal drills
- One borer
- One planer

Each product has a defined profit contribution per unit sold (defined as the sales price per unit minus the cost of raw materials). In addition, the manufacturing of each product requires a certain amount of time on each machine (in hours). The profit and manufacturing time value are shown below. A dash indicates that the manufacturing process for the given product does not require that machine.

| <i></i> | PROD1 | PROD2 | PROD3 | PROD4 | PROD5 | PROD6 | PROD7 |
| --- | --- | --- | --- | --- | --- | --- | --- |
| Profit | 10 | 6 | 8 | 4 | 11 | 9 | 3 |
| Grinding | 0.5 | 0.7 | - | - | 0.3 | 0.2 | 0.5 |
| Vertical Drilling | 0.1 | 0.2 | - | 0.3 | - | 0.6 | - |
| Horizontal Drilling | 0.2 | - | 0.8 | - | - | - | 0.6 |
| Boring | 0.05 | 0.03 | - | 0.07 | 0.1 | - | 0.08 |
| Planning | - | - | 0.01 | - | 0.05 | - | 0.05 |

Instead of pre-defining a maintenance schedule for the machines, as was done in Factory Planning I, in this version of the model we will also optimize the maintenance schedule.

The maintenance requirements are as follows:

- Each machine must be down for maintenance in one month of the six.
- The exception to the above are the grinding machines as only two of them need to be down during the six months.

There are limitations on how many of each product can be sold in a given month. These limits are shown below:

| Month | PROD1 | PROD2 | PROD3 | PROD4 | PROD5 | PROD6 | PROD7 |
| --- | --- | --- | --- | --- | --- | --- | --- |
| January | 500 | 1000 | 300 | 300 | 800 | 200 | 100 |
| February | 600 | 500 | 200 | 0 | 400 | 300 | 150 |
| March | 300 | 600 | 0 | 0 | 500 | 400 | 100 |
| April | 200 | 300 | 400 | 500 | 200 | 0 | 100 |
| May | 0 | 100 | 500 | 100 | 1000 | 300 | 0 |
| June | 500 | 500 | 100 | 300 | 1100 | 500 | 60 |

Up to 100 units of each product may be stored in inventory at a cost of $0.50 per unit per month. At the start of January, there is no product inventory. However, by the end of June, there should be 50 units of each product in inventory.

The factory produces products six days a week using two eight-hour shifts per day. It may be assumed that each month consists of 24 working days. Also, for the purposes of this model, there are no production sequencing issues that need to be taken into account.

What should the production and maintenance plans look like? Also, is it possible to recommend any price increases and determine the value of acquiring any new machines?

This problem is based on a larger model built for the Cornish engineering company of Holman Brothers.

---
## Model Formulation

### Sets and Indices

$t \in \text{Months}=\{\text{Jan},\text{Feb},\text{Mar},\text{Apr},\text{May},\text{Jun}\}$: Set of months.

$p \in \text{Products}=\{1,2,\dots,7\}$: Set of products.

$m \in \text{Machines}=\{\text{Grinder},\text{VertDrill},\text{horiDrill},\text{Borer},\text{Planer}\}$: Set of machines.

### Parameters

$\text{hours_per_month} \in \mathbb{R}^+$: Time (in hours/month) available at any machine on a monthly basis. It results from multiplying the number of working days (24 days) by the number of shifts per day (2) by the duration of a shift (8 hours).

$\text{max_inventory} \in \mathbb{N}$: Maximum number of units of a single product type that can be stored in inventory at any given month.

$\text{holding_cost} \in \mathbb{R}^+$: Monthly cost (in USD/unit/month) of keeping in inventory a unit of any product type.

$\text{store_target} \in \mathbb{N}$: Number of units of each product type to keep in inventory at the end of the planning horizon.

$\text{profit}_p \in \mathbb{R}^+$: Profit (in USD/unit) of product $p$.

$\text{installed}_m \in \mathbb{N}$: Number of machines of type $m$ installed in the factory.

$\text{down_req}_{m} \in \mathbb{N}$: Number of machines of type $m$ that should be scheduled for maintenance at some point in the planning horizon.

$\text{time_req}_{m,p} \in \mathbb{R}^+$: Time (in hours/unit) needed on machine $m$ to manufacture one unit of product $p$.

$\text{max_sales}_{t,p} \in \mathbb{N}$: Maximum number of units of product $p$ that can be sold at month $t$.

### Decision Variables

$\text{make}_{t,p} \in \mathbb{R}^+$: Number of units of product $p$ to manufacture at month $t$.

$\text{store}_{t,p} \in [0, \text{max_inventory}] \subset \mathbb{R}^+$: Number of units of product $p$ to store at month $t$.

$\text{sell}_{t,p} \in [0, \text{max_sales}_{t,p}] \subset \mathbb{R}^+$: Number of units of product $p$ to sell at month $t$.

$\text{repair}_{t,m} \in \{0,1,\dots, \text{down_req}_m\} \subset \mathbb{N}$: Number of machines of type $m$ scheduled for maintenance at month $t$.

**Assumption:** We can produce fractional units.

### Objective Function

- **Profit:** Maximize the total profit (in USD) of the planning horizon.

\begin{equation}
\text{Maximize} \quad Z = \sum_{t \in \text{Months}}\sum_{p \in \text{Products}}
(\text{profit}_p*\text{make}_{t,p} - \text{holding_cost}*\text{store}_{t,p})
\tag{0}
\end{equation}

### Constraints

- **Initial Balance:** For each product $p$, the number of units produced should be equal to the number of units sold plus the number stored (in units of product).

\begin{equation}
\text{make}_{\text{Jan},p} = \text{sell}_{\text{Jan},p} + \text{store}_{\text{Jan},p} \quad \forall p \in \text{Products}
\tag{1}
\end{equation}

- **Balance:** For each product $p$, the number of units produced in month $t$ and previously stored should be equal to the number of units sold and stored in that month (in units of product).

\begin{equation}
\text{store}_{t-1,p} + \text{make}_{t,p} = \text{sell}_{t,p} + \text{store}_{t,p} \quad \forall (t,p) \in \text{Months} \setminus \{\text{Jan}\} \times \text{Products}
\tag{2}
\end{equation}

- **Inventory Target:** The number of units of product $p$ kept in inventory at the end of the planning horizon should hit the target (in units of product).

\begin{equation}
\text{store}_{\text{Jun},p} = \text{store_target} \quad \forall p \in \text{Products}
\tag{3}
\end{equation}

- **Machine Capacity:** Total time used to manufacture any product at machine type $m$ cannot exceed its monthly capacity (in hours).

\begin{equation}
\sum_{p \in \text{Products}}\text{time_req}_{m,p}*\text{make}_{t,p} \leq \text{hours_per_month}*(\text{installed}_m - \text{repair}_{t,m}) \quad \forall (t,m) \in \text{Months} \times \text{Machines}
\tag{4}
\end{equation}

- **Maintenance**: The number of machines of type $m$ scheduled for maintenance should meet the requirement.

\begin{equation}
\sum_{t \in \text{Months}}\text{repair}_{t,m} = \text{down_req}_m \quad \forall m \in \text{Machines}
\tag{5}
\end{equation}

---
## Python Implementation

We import the Gurobi Python Module and other Python libraries.

In [ ]:
%pip install gurobipy

In [1]:
import numpy as np
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.11 & Gurobi 11.0

## Input Data
We define all the input data of the model.

In [2]:
# Parameters

products = ["Prod1", "Prod2", "Prod3", "Prod4", "Prod5", "Prod6", "Prod7"]
machines = ["grinder", "vertDrill", "horiDrill", "borer", "planer"]
months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun"]

profit = {"Prod1":10, "Prod2":6, "Prod3":8, "Prod4":4, "Prod5":11, "Prod6":9, "Prod7":3}

time_req = {
    "grinder": {    "Prod1": 0.5, "Prod2": 0.7, "Prod5": 0.3,
                    "Prod6": 0.2, "Prod7": 0.5 },
    "vertDrill": {  "Prod1": 0.1, "Prod2": 0.2, "Prod4": 0.3,
                    "Prod6": 0.6 },
    "horiDrill": {  "Prod1": 0.2, "Prod3": 0.8, "Prod7": 0.6 },
    "borer": {      "Prod1": 0.05,"Prod2": 0.03,"Prod4": 0.07,
                    "Prod5": 0.1, "Prod7": 0.08 },
    "planer": {     "Prod3": 0.01,"Prod5": 0.05,"Prod7": 0.05 }
}

# number of each machine available
installed = {"grinder":4, "vertDrill":2, "horiDrill":3, "borer":1, "planer":1} 

# number of machines that need to be under maintenance
down_req = {"grinder":2, "vertDrill":2, "horiDrill":3, "borer":1, "planer":1} 

# market limitation of sells
max_sales = {
    ("Jan", "Prod1") : 500,
    ("Jan", "Prod2") : 1000,
    ("Jan", "Prod3") : 300,
    ("Jan", "Prod4") : 300,
    ("Jan", "Prod5") : 800,
    ("Jan", "Prod6") : 200,
    ("Jan", "Prod7") : 100,
    ("Feb", "Prod1") : 600,
    ("Feb", "Prod2") : 500,
    ("Feb", "Prod3") : 200,
    ("Feb", "Prod4") : 0,
    ("Feb", "Prod5") : 400,
    ("Feb", "Prod6") : 300,
    ("Feb", "Prod7") : 150,
    ("Mar", "Prod1") : 300,
    ("Mar", "Prod2") : 600,
    ("Mar", "Prod3") : 0,
    ("Mar", "Prod4") : 0,
    ("Mar", "Prod5") : 500,
    ("Mar", "Prod6") : 400,
    ("Mar", "Prod7") : 100,
    ("Apr", "Prod1") : 200,
    ("Apr", "Prod2") : 300,
    ("Apr", "Prod3") : 400,
    ("Apr", "Prod4") : 500,
    ("Apr", "Prod5") : 200,
    ("Apr", "Prod6") : 0,
    ("Apr", "Prod7") : 100,
    ("May", "Prod1") : 0,
    ("May", "Prod2") : 100,
    ("May", "Prod3") : 500,
    ("May", "Prod4") : 100,
    ("May", "Prod5") : 1000,
    ("May", "Prod6") : 300,
    ("May", "Prod7") : 0,
    ("Jun", "Prod1") : 500,
    ("Jun", "Prod2") : 500,
    ("Jun", "Prod3") : 100,
    ("Jun", "Prod4") : 300,
    ("Jun", "Prod5") : 1100,
    ("Jun", "Prod6") : 500,
    ("Jun", "Prod7") : 60,
}

holding_cost = 0.5
max_inventory = 100
store_target = 50
hours_per_month = 2*8*24

## Model Deployment
We create a model and the variables. We set the UpdateMode parameter to 1 (which simplifies the code – see the documentation for more details). For each product (seven kinds of products) and each time period (month), we will create variables for the amount of which products will get manufactured, held, and sold. In each month, there is an upper limit on the amount of each product that can be sold. This is due to market limitations. For each type of machine and each month we create a variable d, which tells us how many machines are down in this month of this type.

In [3]:
factory = gp.Model('Factory Planning II')

make = factory.addVars(months, products, name="Make") # quantity manufactured
store = factory.addVars(months, products, ub=max_inventory, name="Store") # quantity stored
sell = factory.addVars(months, products, ub=max_sales, name="Sell") # quantity sold
repair = factory.addVars(months, machines, vtype=GRB.INTEGER, ub=down_req, name="Repair") # number of machines down

Using license file c:\gurobi\gurobi.lic


Next, we insert the constraints.
The balance constraints ensure that the amount of product that is in the storage in the prior month and the amount that gets manufactured equals the amount that is sold and held for each product in the current month. This ensures that all products in the model are manufactured in some month. The initial storage is empty.

In [4]:
#1. Initial Balance
Balance0 = factory.addConstrs((make[months[0], product] == sell[months[0], product] 
                  + store[months[0], product] for product in products), name="Initial_Balance")
    
#2. Balance
Balance = factory.addConstrs((store[months[months.index(month) -1], product] + 
                make[month, product] == sell[month, product] + store[month, product] 
                for product in products for month in months 
                if month != months[0]), name="Balance")

The endstore constraints force that at the end of the last month the storage contains the specified amount of each product.

In [5]:
#3. Inventory Target
TargetInv = factory.addConstrs((store[months[-1], product] == store_target for product in products),  name="End_Balance")

The capacity constraints ensure that for each month the time all products require on a certain kind of machine is lower or equal than the available hours for that machine in that month multiplied by the number of available machines in that month. Each product requires some machine hours on different machines. Each machine is down in one or more months due to maintenance, so the number and types of available machines varies per month. There can be multiple machines per machine type.

In [6]:
#4. Machine Capacity
MachineCap = factory.addConstrs((gp.quicksum(time_req[machine][product] * make[month, product]
                             for product in time_req[machine])
                    <= hours_per_month * (installed[machine] - repair[month, machine])
                    for machine in machines for month in months),
                   name = "Capacity")

The maintenance constraints ensure that the specified number and types of machines are down due maintenance in some month. Which month a machine is down is now part of the optimization.

In [7]:
#5. Maintenance

Maintenance = factory.addConstrs((repair.sum('*', machine) == down_req[machine] for machine in machines), "Maintenance")

The objective is to maximize the profit of the company, which consists of the profit for each product minus cost for storing the unsold products. This can be stated as:

In [8]:
#0. Objective Function
obj = gp.quicksum(profit[product] * sell[month, product] -  holding_cost * store[month, product]  
               for month in months for product in products)

factory.setObjective(obj, GRB.MAXIMIZE)

Next, we start the optimization and Gurobi finds the optimal solution.

In [9]:
factory.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 84 rows, 156 columns and 348 nonzeros
Model fingerprint: 0x3c8ea5bf
Variable types: 126 continuous, 30 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-02, 4e+02]
  Objective range  [5e-01, 1e+01]
  Bounds range     [6e+01, 1e+03]
  RHS range        [1e+00, 2e+03]
Found heuristic solution: objective -175.0000000
Presolve removed 22 rows and 27 columns
Presolve time: 0.00s
Presolved: 62 rows, 129 columns, 278 nonzeros
Variable types: 105 continuous, 24 integer (12 binary)

Root relaxation: objective 1.164550e+05, 13 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 116455.000    0   11 -175.00000 116455.000      -     -    0s
H    0     0                    92755.000000 116455.000  

---
## Analysis

The result of the optimization model shows that the maximum profit we can achieve is $\$108,855.00$. This is an increase of $\$15,139.82$ over the course of six months compared to the Factory Planning I example as a result of being able to pick the maintenance schedule as opposed to having a fixed one. Let's see the solution that achieves that optimal result. 

### Production Plan
This plan determines the amount of each product to make at each period of the planning horizon. For example, in February we make 600 units of product Prod1.

In [10]:
rows = months.copy()
columns = products.copy()
make_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for month, product in make.keys():
    if (abs(make[month, product].x) > 1e-6):
        make_plan.loc[month, product] = np.round(make[month, product].x, 1)
make_plan

,Prod1,Prod2,Prod3,Prod4,Prod5,Prod6,Prod7
Jan,500.0,1000.0,300.0,300.0,800.0,200.0,100.0
Feb,600.0,500.0,200.0,0.0,400.0,300.0,150.0
Mar,400.0,700.0,100.0,100.0,600.0,400.0,200.0
Apr,0.0,0.0,0.0,0.0,0.0,0.0,0.0
May,0.0,100.0,500.0,100.0,1000.0,300.0,0.0
Jun,550.0,550.0,150.0,350.0,1150.0,550.0,110.0


### Sales Plan
This plan defines the amount of each product to sell at each period of the planning horizon. For example, in February we sell 600 units of product Prod1.

In [11]:
rows = months.copy()
columns = products.copy()
sell_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for month, product in sell.keys():
    if (abs(sell[month, product].x) > 1e-6):
        sell_plan.loc[month, product] = np.round(sell[month, product].x, 1)
sell_plan

,Prod1,Prod2,Prod3,Prod4,Prod5,Prod6,Prod7
Jan,500.0,1000.0,300.0,300.0,800.0,200.0,100.0
Feb,600.0,500.0,200.0,0.0,400.0,300.0,150.0
Mar,300.0,600.0,0.0,0.0,500.0,400.0,100.0
Apr,100.0,100.0,100.0,100.0,100.0,0.0,100.0
May,0.0,100.0,500.0,100.0,1000.0,300.0,0.0
Jun,500.0,500.0,100.0,300.0,1100.0,500.0,60.0


### Inventory Plan
This plan reflects the amount of product in inventory at the end of each period of the planning horizon. For example, at the end of February we have zero units of Prod1 in inventory.

In [12]:
rows = months.copy()
columns = products.copy()
store_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for month, product in store.keys():
    if (abs(store[month, product].x) > 1e-6):
        store_plan.loc[month, product] = np.round(store[month, product].x, 1)
store_plan

,Prod1,Prod2,Prod3,Prod4,Prod5,Prod6,Prod7
Jan,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Feb,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Mar,100.0,100.0,100.0,100.0,100.0,0.0,100.0
Apr,0.0,0.0,0.0,0.0,0.0,0.0,0.0
May,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Jun,50.0,50.0,50.0,50.0,50.0,50.0,50.0


### Maintenance Plan
This plan shows the maintenance plan for each period of the planning horizon. For example, 2 machines of type grinder will be down for maintenance in April.

In [13]:
rows = months.copy()
columns = machines.copy()
repair_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for month, machine in repair.keys():
    if (abs(repair[month, machine].x) > 1e-6):
        repair_plan.loc[month, machine] = repair[month, machine].x
repair_plan

,grinder,vertDrill,horiDrill,borer,planer
Jan,0.0,0.0,1.0,0.0,0.0
Feb,0.0,1.0,0.0,0.0,0.0
Mar,0.0,0.0,0.0,0.0,0.0
Apr,2.0,0.0,0.0,1.0,1.0
May,0.0,1.0,0.0,0.0,0.0
Jun,0.0,0.0,2.0,0.0,0.0


**Note:** If you want to write your solution to a file, rather than print it to the terminal, you can use the model.write() command. An example implementation is:

`factory.write("factory-planning-2-output.sol")`

---
## References

H. Paul Williams, Model Building in Mathematical Programming, fifth edition.

Copyright &copy; 2020 Gurobi Optimization, LLC